In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

MODEL = "gemma:instruct"

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

'Why did the golfer wear two pairs of pants?\n\nAnswer: In case he got a hole in one!'

In [4]:
embedding = embeddings.embed_query("Tell me a joke")

In [6]:
len(embedding)

3072

In [2]:
import pandas as pd
data = pd.read_csv(r"C:\Users\eriki\OneDrive\Documents\all_folder\Job_interviews\Assignments\SparkBeyond\feature_interpretation_examples.csv")

In [3]:
data.head()

,project_name,project_description,feature,interpretation,score
0,AirBNB,Predict average price for AirBnb listings in a...,distinct(slidingSubsets(flatten(amenities of z...,"Some listing in the area include a kitchen, a ...",1.0
1,AirBNB,Predict average price for AirBnb listings in a...,distinct(slidingSubsets(flatten(amenities of z...,"Some houses in the area contain a kitchen, a d...",1.0
2,AirBNB,Predict average price for AirBnb listings in a...,elementCount(flatten(extractKeys(histogram(ame...,More than 3 buildings in the area contain elev...,1.0
3,AirBNB,Predict average price for AirBnb listings in a...,extractKeys(normalizedHistogram(flatten(amenit...,The words 'conditioning' and 'first' appear in...,1.0
4,AirBNB,Predict average price for AirBnb listings in a...,elementCount(slidingSubsets(flatten(amenities ...,The words 'kit' and 'tv' appear together more ...,1.0


In [4]:
def load_and_remove_few_shot_examples(df, score, num_examples=3):
    """
    Load a specified number of few-shot examples from the dataframe based on the given score
    and remove these examples from the original dataframe.
    """
    examples = []
    filtered_df = df[df['score'] == score].head(num_examples)
    indices_to_remove = filtered_df.index.tolist()
    
    for index, row in filtered_df.iterrows():
        example = {
            "project_description": row['project_description'],
            "feature": row['feature'],
            "interpretation": row['interpretation'],
            "score": int(row['score'])
        }
        examples.append(example)
    
    # Remove the selected examples from the original dataframe
    df = df.drop(indices_to_remove)
    
    return examples, df

few_shot_examples_1, data = load_and_remove_few_shot_examples(data, score=1, num_examples=3)

few_shot_examples_0, data = load_and_remove_few_shot_examples(data, score=0, num_examples=3)

few_shot_examples = few_shot_examples_1 + few_shot_examples_0

In [5]:
few_shot_examples[0]

{'project_description': 'Predict average price for AirBnb listings in a zipcode',
 'feature': 'distinct(slidingSubsets(flatten(amenities of zipcode))) contains ("kitchen", "pets", "allowed", "doorman")',
 'interpretation': 'Some listing in the area include a kitchen, a doorman, and allow pets',
 'score': 1}

In [17]:
def score_handling(score):
    if score == 1:
        return 0.9
    if score == 0:
      return 0.1

In [18]:
def create_prompt(project_description, feature, interpretation, examples):
    prompt = """
    Your answer to this prompt should be a single number between 0 and 1. 
    Given the following project description, feature, and its interpretation, 
    score the quality of the interpretation. Examples:\n"
    """
    
    for example in examples:
        prompt += f"Project Description: {example['project_description']}\n"
        prompt += f"Feature: {example['feature']}\nInterpretation: {example['interpretation']}\nScore: {score_handling(example['score'])}\n"
    prompt += f"Project Description: {project_description}\n"
    prompt += f"Feature: {feature}\nInterpretation: {interpretation}\nScore:"
    return prompt

In [19]:
def score_with_llama(row, examples=[]):
    prompt = create_prompt(row['project_description'], row['feature'], 
                           row['interpretation'], examples)
    
    score = model.invoke(prompt)
    
    try:
        return float(score)
    except ValueError:
        return 0  # Handle unexpected output

In [20]:
df_temp = data.head().copy()
df_temp['llama3_zero_shot_score'] = df_temp.apply(lambda row: score_with_llama(row), axis=1)
df_temp['llama3_few_shot_score'] = df_temp.apply(lambda row: score_with_llama(row, few_shot_examples), axis=1)

In [21]:
df_temp.head()

,project_name,project_description,feature,interpretation,score,llama3_zero_shot_score,llama3_few_shot_score
3,AirBNB,Predict average price for AirBnb listings in a...,extractKeys(normalizedHistogram(flatten(amenit...,The words 'conditioning' and 'first' appear in...,1.0,0.8,0.8
4,AirBNB,Predict average price for AirBnb listings in a...,elementCount(slidingSubsets(flatten(amenities ...,The words 'kit' and 'tv' appear together more ...,1.0,0.0,0.6
5,AirBNB,Predict average price for AirBnb listings in a...,elementCount(slidingSubsets(flatten(amenities ...,"A kitchen, a doorman and an elevator are inclu...",1.0,0.6,0.8
6,AirBNB,Predict average price for AirBnb listings in a...,mapKeysToTextList(histogram(flatten(amenities ...,"The words 'detector', 'essentials', 'shampoo',...",1.0,0.9,0.8
7,AirBNB,Predict average price for AirBnb listings in a...,flatten(extractKeys(histogram(amenities of zip...,The amenities of listings in the area include ...,1.0,0.8,0.8


In [22]:
df_temp[['project_description', 'feature', 'interpretation', 
         'score', 'llama3_zero_shot_score', 'llama3_few_shot_score']].head()

,project_description,feature,interpretation,score,llama3_zero_shot_score,llama3_few_shot_score
3,Predict average price for AirBnb listings in a...,extractKeys(normalizedHistogram(flatten(amenit...,The words 'conditioning' and 'first' appear in...,1.0,0.8,0.8
4,Predict average price for AirBnb listings in a...,elementCount(slidingSubsets(flatten(amenities ...,The words 'kit' and 'tv' appear together more ...,1.0,0.0,0.6
5,Predict average price for AirBnb listings in a...,elementCount(slidingSubsets(flatten(amenities ...,"A kitchen, a doorman and an elevator are inclu...",1.0,0.6,0.8
6,Predict average price for AirBnb listings in a...,mapKeysToTextList(histogram(flatten(amenities ...,"The words 'detector', 'essentials', 'shampoo',...",1.0,0.9,0.8
7,Predict average price for AirBnb listings in a...,flatten(extractKeys(histogram(amenities of zip...,The amenities of listings in the area include ...,1.0,0.8,0.8


In [23]:
data['llama3_zero_shot_score'] = data.apply(lambda row: score_with_llama(row), axis=1)
data['llama3_few_shot_score'] = data.apply(lambda row: score_with_llama(row, few_shot_examples), axis=1)


data[['project_description', 'feature', 'interpretation', 
      'score', 'llama3_zero_shot_score', 'llama3_few_shot_score']].head()

,project_description,feature,interpretation,score,llama3_zero_shot_score,llama3_few_shot_score
3,Predict average price for AirBnb listings in a...,extractKeys(normalizedHistogram(flatten(amenit...,The words 'conditioning' and 'first' appear in...,1.0,0.50,0.8
4,Predict average price for AirBnb listings in a...,elementCount(slidingSubsets(flatten(amenities ...,The words 'kit' and 'tv' appear together more ...,1.0,0.25,0.8
5,Predict average price for AirBnb listings in a...,elementCount(slidingSubsets(flatten(amenities ...,"A kitchen, a doorman and an elevator are inclu...",1.0,0.80,0.8
6,Predict average price for AirBnb listings in a...,mapKeysToTextList(histogram(flatten(amenities ...,"The words 'detector', 'essentials', 'shampoo',...",1.0,0.80,0.8
7,Predict average price for AirBnb listings in a...,flatten(extractKeys(histogram(amenities of zip...,The amenities of listings in the area include ...,1.0,0.50,0.9


In [28]:
data = pd.read_csv(r"C:\Users\eriki\OneDrive\Documents\all_folder\Job_interviews\Assignments\SparkBeyond\llama3_with_results.csv")

## Evaluation of performance

In [29]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

llama_zero_shot_mse = mean_squared_error(data['score'], data['llama3_zero_shot_score'])
llama_few_shot_mse = mean_squared_error(data['score'], data['llama3_few_shot_score'])

baseline_rmse = np.sqrt(llama_zero_shot_mse)
llama_rmse = np.sqrt(llama_few_shot_mse)

# Calculate R^2 for both models
baseline_r2 = r2_score(data['score'], data['llama3_zero_shot_score'])
llama_r2 = r2_score(data['score'], data['llama3_few_shot_score'])

print(f"Baseline MSE: {llama_zero_shot_mse}")
print(f"LLaMA MSE: {llama_few_shot_mse}")
print(f"Baseline RMSE: {baseline_rmse}")
print(f"LLaMA RMSE: {llama_rmse}")
print(f"Baseline R²: {baseline_r2}")
print(f"LLaMA R²: {llama_r2}")

Baseline MSE: 0.32920774193548386
LLaMA MSE: 0.4070201612903226
Baseline RMSE: 0.5737662781442316
LLaMA RMSE: 0.6379813173521013
Baseline R²: -0.3168309677419354
LLaMA R²: -0.6280806451612904


### Without seen projects

In [30]:
def get_seen_projects(few_shot_examples):
    return [example['project_description'] for example in few_shot_examples]

seen_projects = get_seen_projects(few_shot_examples)

def subset_unseen_projects(df, seen_projects):
    return df[~df['project_description'].isin(seen_projects)]

unseen_data = subset_unseen_projects(data, seen_projects)

In [31]:
gpt_zero_shot_mse = mean_squared_error(unseen_data['score'], unseen_data['llama3_zero_shot_score'])
gpt_few_shot_mse = mean_squared_error(unseen_data['score'], unseen_data['llama3_few_shot_score'])

gpt_zero_shot_rmse = np.sqrt(gpt_zero_shot_mse)
gpt_few_shot_rmse = np.sqrt(gpt_few_shot_mse)

# Calculate R^2 for both models
zero_shot_r2 = r2_score(unseen_data['score'], unseen_data['llama3_zero_shot_score'])
few_shot_r2 = r2_score(unseen_data['score'], unseen_data['llama3_few_shot_score'])

print(f"Zero Shot MSE: {gpt_zero_shot_mse}")
print(f"Few Shot MSE: {gpt_few_shot_mse}")
print(f"Zero Shot RMSE: {gpt_zero_shot_rmse}")
print(f"Few Shot RMSE: {gpt_few_shot_rmse}")
print(f"Zero Shot R²: {zero_shot_r2}")
print(f"Few Shot R²: {few_shot_r2}")

Zero Shot MSE: 0.32984255319148936
Few Shot MSE: 0.40671440261865793
Zero Shot RMSE: 0.5743192084472618
Few Shot RMSE: 0.6377416425314077
Zero Shot R²: -0.31937021276595745
Few Shot R²: -0.6268576104746317


In [27]:
# Save the data with the LLaMA scores
data.to_csv(r"C:\Users\eriki\OneDrive\Documents\all_folder\Job_interviews\Assignments\SparkBeyond\llama3_with_results_09_01.csv", index=False)